<a href="https://colab.research.google.com/github/BrunoShinniti/transformer-mecanismo-de-atencao/blob/main/Mecanismo_de_Aten%C3%A7%C3%A3o_(Transformer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## com PyTorch

In [5]:
!pip install -q -U torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 118.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [11]:
import torch
from torch import nn

In [31]:
class Transformer(nn.Module):

    def __init__(self, vocab_size, embedding_dim, n_heads, n_layers, dropout):
        super().__init__()

        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.n_heads = n_heads
        self.n_layers = n_layers
        self.drouptout = dropout

        # Define a camada de embedding que transformará a sequência de entrada em uma sequência de vetores de dimensão fixa
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Define o mecanismo de auto-atenção multi-head
        self.attention = nn.MultiheadAttention(embedding_dim, n_heads, dropout = dropout)

        # Define a rede neural feed-forward qe será usada para gerar a sequência de saída a partir da sequência de entrada
        self.feed_forward = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(embedding_dim, embedding_dim)
        )

        # Define a camada de saída final que transformará a sequência de saída na forma de saída desejada
        self.out = nn.Linear(embedding_dim, vocab_size)


    def forward(self, x):

        # Aplica a camada de embedding à sequência de entrada
        x = self.embedding(x)

        # Aplica o mecanismo multi-headed self-attetion
        x = self.attention(x)

        # Aplica o método feed-forward
        x = self.feed_forward(x)

        # Aplica camada final
        x = self.out(x)

        return x


modelo = Transformer(vocab_size = 1000,
                     embedding_dim = 32,
                     n_heads = 4,
                     n_layers = 2,
                     dropout = 0.5)

modelo.modules

<bound method Module.modules of Transformer(
  (embedding): Embedding(1000, 32)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
  )
  (feed_forward): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (out): Linear(in_features=32, out_features=1000, bias=True)
)>

In [33]:
modelo.attention

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
)

## Sem Framework

Construção de modelo capaz de prever sequências de comprimento igual a 10 tokens.

* 1 - Camada de Embedding: Transforma as palavras em vetores numéricos de tamanho fixo.

* 2 - Mecanismo de Atenção: Permite que o modelo foque em diferentes partes da entrada.

* 3 - Camadas Encoder e Decoder: Processam os dados sequencialmente

* 4 - Camada Linear e Softmax: Para predições finais

#### Objetivo:
Implementar o item 2, mas para deixar o exemplo funcional, será implementado os itens 1 e 4.

### Hiperparâmetros Iniciais

In [34]:
import numpy as np

In [38]:
# Dimensão do modelo
dim_model = 64

In [37]:
# Comprimento da sequência
seq_lenght = 10